# Utilitários

## Copia arquivo para servidor remoto

### Instala sshpass para passar a senha para o scp

In [ ]:
!apt install sshpass

### Executa scp para copiar arquivo

In [ ]:
user = ""
senha = ""
host = "rafaelescalfoni.net"
file = "prefeito_rio_2020_clean.csv"
# Executa comando no bash
!sshpass -p $senha scp -o 'StrictHostKeyChecking no' $file $user@$host:

# Bibliotecas

In [ ]:
import pandas as pd
import json
# Imports individuais
from pandas.core.frame import DataFrame

# Abre o DataSet diretamente da internet

Desabilita verificação SSL em caso de certificado auto-assinado:

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

Lê o Dataset Original:

In [ ]:
url = "https://rafaelescalfoni.net/prefeito_rio_2020.csv"
ds = pd.read_csv(url)
ds.info()

Lê o Dataset Limpo:

In [ ]:
url = "https://rafaelescalfoni.net/prefeito_rio_2020_clean.csv"
# Parâmetro "na_filter" usado para lidar com campos nulos como string
ds = pd.read_csv(url, na_filter=False)
ds.info()

# Limpeza de Dados

## Eliminação de colunas

### Função para eliminar colunas com taxa de nulos acima do limite

In [ ]:
def drop_column_null_above_limit(df: DataFrame, threshold=0.8):
  # Pega colunas acima do limite de null
  columns = [column for column in df.columns if df[column].isna().sum()/df.shape[0] > threshold]

  return df.drop(columns, axis=1)

### Elimina colunas com alta taxa de nulos

In [ ]:
ds = drop_column_null_above_limit(ds)
ds.info()

### Função para eliminar colunas com um único valor

In [ ]:
def drop_column_unique_value(df: DataFrame):
  # Pega colunas com um único valor
  columns = [column for column in df.columns if df[column].nunique() == 1]

  return df.drop(columns, axis=1)

### Elimina colunas com um único valor

In [ ]:
ds = drop_column_unique_value(ds)
ds.info()

### Elimina as colunas que não têm utilidade para nossa análise

In [ ]:
ignored_columns = ['_id', 'id_str']
ds.drop(columns=ignored_columns, inplace=True)
ds.info()

## Tratamento das colunas JSON

### Lista colunas com objetos JSON

In [ ]:
# Função que verifica se determinada string é um objeto json (parâmetro opcional = somente dicionário)
def is_json(myjson, ignore_list=True):
  try:
    json_object = json.loads(myjson)
  except:
    return False
  # Verifica se o objeto json é uma lista
  if (ignore_list and type(json_object) == list):
    return False
  else:
    return True
# Analisa sempre o primeiro índice não nulo da referida coluna, ou 0 quando a coluna é vazia
[column for column in ds.columns if is_json(ds[column][ds[column].first_valid_index() or 0])]

### Expandir colunas JSON

#### Coluna user

Transforma os objetos JSON da coluna em um DataFrame:

In [ ]:
expanded_user = pd.json_normalize(ds.user.apply(json.loads))
expanded_user.info()

Elimina colunas com alta taxa de nulos:

In [ ]:
expanded_user = drop_column_null_above_limit(expanded_user)
expanded_user.info()

Elimina colunas com um único valor:

In [ ]:
expanded_user = drop_column_unique_value(expanded_user)
expanded_user.info()

Elimina as colunas que não têm utilidade para nossa análise:

In [ ]:
ignored_columns = ['id', 'url', 'translator_type', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https',
                   'profile_background_tile', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color',
                   'profile_use_background_image', 'profile_image_url', 'profile_image_url_https', 'profile_banner_url', 'default_profile', 'id.$numberLong']
expanded_user.drop(columns=ignored_columns, inplace=True)
expanded_user.info()

#### Coluna entities

Transforma os objetos JSON da coluna em um DataFrame:

In [ ]:
expanded_entities = pd.json_normalize(ds.entities.apply(json.loads))
expanded_entities.info()

Elimina colunas com alta taxa de nulos:

In [ ]:
expanded_entities = drop_column_null_above_limit(expanded_entities)
expanded_entities.info()

Elimina as colunas que não têm utilidade para nossa análise:

In [ ]:
ignored_columns = ['symbols']
expanded_entities.drop(columns=ignored_columns, inplace=True)
expanded_entities.info()

## Faz o Join desses datasets com o principal

In [ ]:
# Adiciona prefixo "user_" às colunas do DS "expanded_user" antes do join
ds = ds.join(expanded_user.add_prefix('user_'))
# Adiciona prefixo "entities_" às colunas do DS "expanded_entities" antes do join
ds = ds.join(expanded_entities.add_prefix('entities_'))
# Elimina as colunas JSON
ds.drop(columns=['user', 'entities'], inplace=True)
ds.info()

## Preenchimento de nulos

In [ ]:
# Preenche campos texto com a string vazia
ds['source'] = ds['source'].fillna('')
ds['user_location'] = ds['user_location'].fillna('')
ds['user_description'] = ds['user_description'].fillna('')

## Salva Dataset limpo

In [ ]:
# Parâmetro "escapechar" usado para lidar com quebra de linhas nos campos texto
ds.to_csv('prefeito_rio_2020_clean.csv', index=False, escapechar='\r')

# TODO Processamento de Texto

In [ ]:
# https://towardsai.net/p/data-mining/text-mining-in-python-steps-and-examples-78b3f8fd913b
# https://machinelearningmastery.com/clean-text-machine-learning-python/
# Cria datasets em portugês e inglês
ds_por = ds[ds.language == 'portuguese']
ds_ing = ds[ds.language == 'english']
ds_ing

In [ ]:
import pandas as pd
import nltk

nltk.download('punkt')
df = pd.DataFrame({'frases': ['Do not let the day end without having grown a little,', 'without having been happy, without having increased your dreams', 'Do not let yourself be overcomed by discouragement.','We are passion-full beings.']})
df['tokenized'] = df.apply(lambda row: nltk.word_tokenize(row['frases']), axis=1)
df

# TODO Análise Exploratória de Dados

In [ ]:
ds['language'].value_counts()